1. In this kernel we will be creating a database file from the json data.Also we will covers topics related to doing queries with SQLite.This kernel is work in process and if you like the kernel please do vote.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movies-sql/some_movies.json


# Creating an SQL Database(Movies Dataset)

In [2]:
import sqlite3 as sq3

### Creating a new Database and connection to the database

In [3]:
con = sq3.connect("movies.db")

In [4]:
con

In [5]:
# Checking tables in the newly created database movies
con.execute("Select * FROM sqlite_master").fetchall()

[]

As expected there are no tables in out database.

In [6]:
# Getting List of Table names 
con.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name").fetchall()

[]

As we dont have any tables in database we didnt get the names of the tables.

In [7]:
# Closing connection
con.close()

# Loading Data from DataFrames into an SQLite Database

In [8]:
import pandas as pd
import json
import sqlite3 as sq3

In [9]:
with open("../input/movies-sql/some_movies.json") as f:
    data = json.load(f)

In [10]:
#data.head(1)

In [11]:
# Conveting the json file to pandas dataframe
df = pd.json_normalize(data,sep = "_")
df.head()

,adult,backdrop_path,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,tagline,title,video,vote_average,vote_count,belongs_to_collection_id,belongs_to_collection_name,belongs_to_collection_poster_path,belongs_to_collection_backdrop_path,belongs_to_collection
0,False,/orjiB3oUIsyz60hoEqkiGpy5CeO.jpg,356000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 878, ...",https://www.marvel.com/movies/avengers-endgame,299534,tt4154796,en,Avengers: Endgame,After the devastating events of Avengers: Infi...,...,Part of the journey is the end.,Avengers: Endgame,False,8.3,12607,86311.0,The Avengers Collection,/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg,/zuW6fOiusv4X9nnW3paHGfXcSll.jpg,NaN
1,False,/wcC7kCICL6x6zHUlUyNp9pWoqW1.jpg,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.avatarmovie.com/,19995,tt0499549,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",...,Enter the World of Pandora.,Avatar,False,7.4,21000,87096.0,Avatar Collection,/nslJVsO58Etqkk17oXMuVK4gNOF.jpg,/8nCr9W7sKus2q9PLbYsnT7iCkuT.jpg,NaN
2,False,/8BTsTfln4jlQrLXUBquXJ0ASQy9.jpg,245000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.starwars.com/films/star-wars-episod...,140607,tt2488496,en,Star Wars: The Force Awakens,Thirty years after defeating the Galactic Empi...,...,Every generation has a story.,Star Wars: The Force Awakens,False,7.4,14205,10.0,Star Wars Collection,/r8Ph5MYXL04Qzu4QBbq2KjqwtkQ.jpg,/d8duYyyC9J5T825Hg7grmaabfxQ.jpg,NaN
3,False,/lmZFxXgJE3vgrciwuDib0N8CfQo.jpg,300000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",https://www.marvel.com/movies/avengers-infinit...,299536,tt4154756,en,Avengers: Infinity War,As the Avengers and their allies have continue...,...,An entire universe. Once and for all.,Avengers: Infinity War,False,8.3,17718,86311.0,The Avengers Collection,/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg,/zuW6fOiusv4X9nnW3paHGfXcSll.jpg,NaN
4,False,/6VmFqApQRyZZzmiGOQq2C92jyvH.jpg,200000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",,597,tt0120338,en,Titanic,101-year-old Rose DeWitt Bukater tells the sto...,...,Nothing on Earth could come between them.,Titanic,False,7.8,16661,NaN,NaN,NaN,NaN,NaN


### Creating a dataframe movies,votes,generes and prod with important features

In [12]:
# Creating a dataframe movies with important features
movies = df[["id","title","budget","revenue","belongs_to_collection_name","release_date"]].copy()

In [13]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   id                          18 non-null     int64 
 1   title                       18 non-null     object
 2   budget                      18 non-null     int64 
 3   revenue                     18 non-null     int64 
 4   belongs_to_collection_name  15 non-null     object
 5   release_date                18 non-null     object
dtypes: int64(3), object(3)
memory usage: 992.0+ bytes


Here the release data should be converted from Object to Datatime type. We will also convert the revenue and budget in terms of millions of usa dollar.

In [14]:
movies.release_date = pd.to_datetime(df.release_date)

In [15]:
movies.revenue = df.revenue/1000000
movies.budget = df.budget/1000000
movies.head()

,id,title,budget,revenue,belongs_to_collection_name,release_date
0,299534,Avengers: Endgame,356.0,2797.800564,The Avengers Collection,2019-04-24
1,19995,Avatar,237.0,2787.965087,Avatar Collection,2009-12-10
2,140607,Star Wars: The Force Awakens,245.0,2068.223624,Star Wars Collection,2015-12-15
3,299536,Avengers: Infinity War,300.0,2046.239637,The Avengers Collection,2018-04-25
4,597,Titanic,200.0,1845.034188,NaN,1997-11-18


In [16]:
votes = df[['id','vote_count','vote_average']].copy()
votes.head()

,id,vote_count,vote_average
0,299534,12607,8.3
1,19995,21000,7.4
2,140607,14205,7.4
3,299536,17718,8.3
4,597,16661,7.8


In [17]:
genres = pd.json_normalize(data=data,record_path='genres',meta='id',record_prefix='genre_')
genres.head()

,genre_id,genre_name,id
0,12,Adventure,299534
1,878,Science Fiction,299534
2,28,Action,299534
3,28,Action,19995
4,12,Adventure,19995


In [18]:
prod = pd.json_normalize(data=data,record_path='production_companies',meta='id',record_prefix='prod_')
prod.head()

,prod_id,prod_logo_path,prod_name,prod_origin_country,id
0,420,/hUzeosd33nzE5MCNsZxCGEKTXaQ.png,Marvel Studios,US,299534
1,444,/42UPdZl6B2cFXgNUASR8hSt9mpS.png,Dune Entertainment,US,19995
2,574,/iB6GjNVHs5hOqcEYt2rcjBqIjki.png,Lightstorm Entertainment,US,19995
3,25,/qZCc1lty5FzX30aOCVRBLzaVmcp.png,20th Century Fox,US,19995
4,290,/Q8mw2AOQQc8Qg0uNwLWq86DVZv.png,Ingenious Media,GB,19995


In [19]:
# Connecting to data base
con = sq3.connect("movies.db")

In [20]:
con

### Creating Tables

In [21]:
movies.to_sql("Movies",con,index=False)

In [22]:
votes.to_sql("Votes",con,index=False)

In [23]:
genres.to_sql("Genres",con,index=False)

In [24]:
prod.to_sql("Prod",con,index=False)

In [25]:
# Getting the table details 
con.execute("Select * FROM sqlite_master").fetchall()

[('table',
  'Movies',
  'Movies',
  2,
  'CREATE TABLE "Movies" (\n"id" INTEGER,\n  "title" TEXT,\n  "budget" REAL,\n  "revenue" REAL,\n  "belongs_to_collection_name" TEXT,\n  "release_date" TIMESTAMP\n)'),
 ('table',
  'Votes',
  'Votes',
  3,
  'CREATE TABLE "Votes" (\n"id" INTEGER,\n  "vote_count" INTEGER,\n  "vote_average" REAL\n)'),
 ('table',
  'Genres',
  'Genres',
  4,
  'CREATE TABLE "Genres" (\n"genre_id" INTEGER,\n  "genre_name" TEXT,\n  "id" INTEGER\n)'),
 ('table',
  'Prod',
  'Prod',
  5,
  'CREATE TABLE "Prod" (\n"prod_id" INTEGER,\n  "prod_logo_path" TEXT,\n  "prod_name" TEXT,\n  "prod_origin_country" TEXT,\n  "id" INTEGER\n)')]

In [26]:
# Getting the table names
con.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name").fetchall()

[('Genres',), ('Movies',), ('Prod',), ('Votes',)]

In [27]:
con.close()

# Loading Data from SQLite Database into DataFrames

In [28]:
import pandas as pd
import sqlite3 as sq3

In [29]:
con = sq3.connect("movies.db")

In [30]:
#con.execute("Select * FROM sqlite_master").fetchall()

In [31]:
# Getting table info
pd.read_sql("SELECT * FROM Movies",con)

,id,title,budget,revenue,belongs_to_collection_name,release_date
0,299534,Avengers: Endgame,356.0,2797.800564,The Avengers Collection,2019-04-24 00:00:00
1,19995,Avatar,237.0,2787.965087,Avatar Collection,2009-12-10 00:00:00
2,140607,Star Wars: The Force Awakens,245.0,2068.223624,Star Wars Collection,2015-12-15 00:00:00
3,299536,Avengers: Infinity War,300.0,2046.239637,The Avengers Collection,2018-04-25 00:00:00
4,597,Titanic,200.0,1845.034188,None,1997-11-18 00:00:00
5,135397,Jurassic World,150.0,1671.713208,Jurassic Park Collection,2015-06-06 00:00:00
6,420818,The Lion King,260.0,1656.943394,None,2019-07-12 00:00:00
7,24428,The Avengers,220.0,1519.557910,The Avengers Collection,2012-04-25 00:00:00
8,168259,Furious 7,190.0,1515.047671,The Fast and the Furious Collection,2015-04-01 00:00:00
9,99861,Avengers: Age of Ultron,250.0,1405.403694,The Avengers Collection,2015-04-22 00:00:00


### Converting table to Pandas data frame 

In [32]:
df = pd.read_sql("SELECT * FROM Movies",con,index_col='id',parse_dates='release_date')
df.head()

,title,budget,revenue,belongs_to_collection_name,release_date
id,,,,,
299534,Avengers: Endgame,356.0,2797.800564,The Avengers Collection,2019-04-24
19995,Avatar,237.0,2787.965087,Avatar Collection,2009-12-10
140607,Star Wars: The Force Awakens,245.0,2068.223624,Star Wars Collection,2015-12-15
299536,Avengers: Infinity War,300.0,2046.239637,The Avengers Collection,2018-04-25
597,Titanic,200.0,1845.034188,None,1997-11-18


In [33]:
#df.info()

In [34]:
genres = pd.read_sql("SELECT * FROM Genres",con,index_col='id')
genres.head()

,genre_id,genre_name
id,,
299534,12,Adventure
299534,878,Science Fiction
299534,28,Action
19995,28,Action
19995,12,Adventure


In [35]:
con.close()

# Simple SQL Queries

In [36]:
import pandas as pd
import sqlite3 as sq3

In [37]:
con = sq3.connect("movies.db")

In [38]:
pd.read_sql("SELECT * FROM Movies",con)

,id,title,budget,revenue,belongs_to_collection_name,release_date
0,299534,Avengers: Endgame,356.0,2797.800564,The Avengers Collection,2019-04-24 00:00:00
1,19995,Avatar,237.0,2787.965087,Avatar Collection,2009-12-10 00:00:00
2,140607,Star Wars: The Force Awakens,245.0,2068.223624,Star Wars Collection,2015-12-15 00:00:00
3,299536,Avengers: Infinity War,300.0,2046.239637,The Avengers Collection,2018-04-25 00:00:00
4,597,Titanic,200.0,1845.034188,None,1997-11-18 00:00:00
5,135397,Jurassic World,150.0,1671.713208,Jurassic Park Collection,2015-06-06 00:00:00
6,420818,The Lion King,260.0,1656.943394,None,2019-07-12 00:00:00
7,24428,The Avengers,220.0,1519.557910,The Avengers Collection,2012-04-25 00:00:00
8,168259,Furious 7,190.0,1515.047671,The Fast and the Furious Collection,2015-04-01 00:00:00
9,99861,Avengers: Age of Ultron,250.0,1405.403694,The Avengers Collection,2015-04-22 00:00:00


In [39]:
# Following the SQL code Standard of writing codes in different line
pd.read_sql("SELECT * \
             FROM Movies",con)

,id,title,budget,revenue,belongs_to_collection_name,release_date
0,299534,Avengers: Endgame,356.0,2797.800564,The Avengers Collection,2019-04-24 00:00:00
1,19995,Avatar,237.0,2787.965087,Avatar Collection,2009-12-10 00:00:00
2,140607,Star Wars: The Force Awakens,245.0,2068.223624,Star Wars Collection,2015-12-15 00:00:00
3,299536,Avengers: Infinity War,300.0,2046.239637,The Avengers Collection,2018-04-25 00:00:00
4,597,Titanic,200.0,1845.034188,None,1997-11-18 00:00:00
5,135397,Jurassic World,150.0,1671.713208,Jurassic Park Collection,2015-06-06 00:00:00
6,420818,The Lion King,260.0,1656.943394,None,2019-07-12 00:00:00
7,24428,The Avengers,220.0,1519.557910,The Avengers Collection,2012-04-25 00:00:00
8,168259,Furious 7,190.0,1515.047671,The Fast and the Furious Collection,2015-04-01 00:00:00
9,99861,Avengers: Age of Ultron,250.0,1405.403694,The Avengers Collection,2015-04-22 00:00:00


In [40]:
# Selecting specific columns of the table
pd.read_sql("SELECT id,revenue,release_date FROM Movies",con)

,id,revenue,release_date
0,299534,2797.800564,2019-04-24 00:00:00
1,19995,2787.965087,2009-12-10 00:00:00
2,140607,2068.223624,2015-12-15 00:00:00
3,299536,2046.239637,2018-04-25 00:00:00
4,597,1845.034188,1997-11-18 00:00:00
5,135397,1671.713208,2015-06-06 00:00:00
6,420818,1656.943394,2019-07-12 00:00:00
7,24428,1519.557910,2012-04-25 00:00:00
8,168259,1515.047671,2015-04-01 00:00:00
9,99861,1405.403694,2015-04-22 00:00:00


In [41]:
# Summing up the revence 
pd.read_sql("SELECT sum(revenue) FROM Movies",con)

,sum(revenue)
0,29748.575327


In [42]:
# Summing up the revence 
con.execute("SELECT sum(revenue) FROM Movies").fetchall()[0][0]

29748.575327000002

In [43]:
# Counting the Distinct entries in title
pd.read_sql("SELECT count(title) FROM Movies",con)

,count(title)
0,18


In [44]:
# Counting number of rows
pd.read_sql("SELECT count(*) FROM Movies",con)

,count(*)
0,18


In [45]:
# Average budget for the 18 movies
pd.read_sql("SELECT avg(budget) FROM Movies",con)

,avg(budget)
0,209.055556


In [46]:
con.close()

# Additional SQL Queries

In [47]:
import pandas as pd
import sqlite3 as sq3

In [48]:
con = sq3.connect('movies.db')

In [49]:
pd.read_sql("SELECT * FROM Movies WHERE id = 597",con)

,id,title,budget,revenue,belongs_to_collection_name,release_date
0,597,Titanic,200.0,1845.034188,None,1997-11-18 00:00:00


In [50]:
pd.read_sql("SELECT * FROM Movies WHERE revenue > 2000",con)

,id,title,budget,revenue,belongs_to_collection_name,release_date
0,299534,Avengers: Endgame,356.0,2797.800564,The Avengers Collection,2019-04-24 00:00:00
1,19995,Avatar,237.0,2787.965087,Avatar Collection,2009-12-10 00:00:00
2,140607,Star Wars: The Force Awakens,245.0,2068.223624,Star Wars Collection,2015-12-15 00:00:00
3,299536,Avengers: Infinity War,300.0,2046.239637,The Avengers Collection,2018-04-25 00:00:00


In [51]:
pd.read_sql("SELECT * FROM Movies WHERE revenue > 1500 AND budget <200",con)

,id,title,budget,revenue,belongs_to_collection_name,release_date
0,135397,Jurassic World,150.0,1671.713208,Jurassic Park Collection,2015-06-06 00:00:00
1,168259,Furious 7,190.0,1515.047671,The Fast and the Furious Collection,2015-04-01 00:00:00


In [52]:
pd.read_sql("SELECT MIN(budget) FROM Movies WHERE revenue > 1250",con)

,MIN(budget)
0,125.0


In [53]:
pd.read_sql("SELECT DISTINCT title FROM Movies",con)

,title
0,Avengers: Endgame
1,Avatar
2,Star Wars: The Force Awakens
3,Avengers: Infinity War
4,Titanic
5,Jurassic World
6,The Lion King
7,The Avengers
8,Furious 7
9,Avengers: Age of Ultron


In [54]:
pd.read_sql("SELECT * FROM Movies ORDER BY budget DESC",con)

,id,title,budget,revenue,belongs_to_collection_name,release_date
0,299534,Avengers: Endgame,356.0,2797.800564,The Avengers Collection,2019-04-24 00:00:00
1,299536,Avengers: Infinity War,300.0,2046.239637,The Avengers Collection,2018-04-25 00:00:00
2,420818,The Lion King,260.0,1656.943394,None,2019-07-12 00:00:00
3,99861,Avengers: Age of Ultron,250.0,1405.403694,The Avengers Collection,2015-04-22 00:00:00
4,140607,Star Wars: The Force Awakens,245.0,2068.223624,Star Wars Collection,2015-12-15 00:00:00
5,19995,Avatar,237.0,2787.965087,Avatar Collection,2009-12-10 00:00:00
6,24428,The Avengers,220.0,1519.557910,The Avengers Collection,2012-04-25 00:00:00
7,597,Titanic,200.0,1845.034188,None,1997-11-18 00:00:00
8,284054,Black Panther,200.0,1346.739107,Black Panther Collection,2018-02-13 00:00:00
9,181808,Star Wars: The Last Jedi,200.0,1332.539889,Star Wars Collection,2017-12-13 00:00:00


In [55]:
pd.read_sql("SELECT * FROM Movies WHERE belongs_to_collection_name is NULL",con)

,id,title,budget,revenue,belongs_to_collection_name,release_date
0,597,Titanic,200.0,1845.034188,None,1997-11-18 00:00:00
1,420818,The Lion King,260.0,1656.943394,None,2019-07-12 00:00:00
2,321612,Beauty and the Beast,160.0,1263.521126,None,2017-03-16 00:00:00


In [56]:
#pd.read_sql("SELECT * FROM Movies WHERE belongs_to_collection_name is IS NOT NULL",con)

In [57]:
pd.read_sql("SELECT * FROM Movies WHERE title LIKE 'Avengers%'",con)

,id,title,budget,revenue,belongs_to_collection_name,release_date
0,299534,Avengers: Endgame,356.0,2797.800564,The Avengers Collection,2019-04-24 00:00:00
1,299536,Avengers: Infinity War,300.0,2046.239637,The Avengers Collection,2018-04-25 00:00:00
2,99861,Avengers: Age of Ultron,250.0,1405.403694,The Avengers Collection,2015-04-22 00:00:00


In [58]:
con.close()